## Multi-class classification

A large part of academic literature and most of the natural definitions of concepts in classification assume that we only have two possible classes, "positive" and "negative".
In reality, however, we have plenty of problems with more than two classes: an OCR reader which needs to map pictures to characters, a classifier of songs into genres, an automatic tagger for pictures, etc.

Ideally, we would still keep using all the machinery we develop for binary classification, treat it as a *black box*, and magically perform multi-class classification with it.
It turns out that, for the (relatively) simple models we consider, this can be done.
The degree of success that one can get when using binary classifiers for multi-class classification varies, and for complex problems, as a rule of thumb, one should look directly at multi-class classifiers (such as tree-based models, or neural networks).
For easy to medium-complexity tasks, however, the approaches that we describe in the following have given good results on many practical problems.

Imagine to have a classifier $\hat{f}$ which returns a continuous score (e.g., a probability or a confidence measure) when trained on a binary classification task.
How can we turn this classifier into something useful for multi-class classification, when we have $K$ different possible classes?
There are two main methods we can use, which we describe in the following.

### 1-vs-1 classification

A natural way to use our binary $\hat{f}$ is to consider the classes two-by-two, and train a different estimator for each pair of classes.
For example, if we want to train a classifier for pictures of either dogs, cats or bunnies, we can actually train three binary classifiers: dog vs. cat, dog vs. bunny, cat vs. bunny.
These three classifiers cover all possible pairs of classes.
In general, for $K$ classes, we would have to train ${K \choose 2}$ classifiers, because there are ${K \choose 2}$ ways of selecting two classes out of $K$.

For any two classes $k_1$ and $k_2$ (with $k_1 \neq k_2$) call $\hat{f}_{k_1, k_2}$ the corresponding classifier.
How can we use the ${K \choose 2}$ classifiers of type $\hat{f}_{k_1, k_2}$ to perform multi-class classification?
The idea is that, given a new data-point $X$, we compute all predictions $\hat{f}_{k_1, k_2}(X)$.
For each class $k$, we record how many times these predictors have assigned class $k$ to point $X$, and we call this number $n_k$.
At the end, we select as our overall predicion the class with the largest $n_k$.
In other words, we assign point $X$ the class which was predicted most often.

The rationale behind this approach is the following.
Let $Y$ be the real class to which $X$ belong.
Most of the times when we compute $\hat{f}_{k, Y}(X)$ (for the other classes $k \neq Y$), we expect that $\hat{f}$ will (correctly) assign class $Y$.
When instead we compute $\hat{f}_{k_1, k_2}$ and neither $k_1$ nor $k_2$ is the real class ($k_1 \neq Y$, $k_2 \neq Y$) the results will tend to be much more affected by uncertainty and randomness.
So, for any other class $k \neq Y$, we expect that it will "win" a duel fewer times than the real class $Y$.

An obsious disadvantage of this method is that we have to train and query ${K \choose 2}$ classifiers
This number can become large quickly.
For example, for an OCR reader which needs to distinguish 26 lower case letter + 26 upper case letter + 10 digits + 5 punctuation signs, we have ${67 \choose 2} = 2211$ classifiers!

### 1-vs-all classification

In 1-vs-all classification (also called 1-vs-rest), we only train $K$ classifiers $\hat{f}_1, \ldots, \hat{f}_K$.
Each classifier $\hat{f}_k$ is used to discriminate between points of class $k$ vs. all other points, belonging to all other classes.
Therefore, each classifier $\hat{f}_k$ outputs a measure of the model's confidence that a point is of class $k$.

When given a new point $X$, we can compute all these confidence scores $\hat{f}_1(X), \ldots, \hat{f}_K(X)$.
In the end, we assign $X$ to the class $k$ which gives the highest score.

The idea here is that, given a point $X$ whose real class is $Y$, the classifier discriminating class $Y$ vs. the rest will be the most confident when placing $X$ in its corresponding class.

Compared to 1-vs-1 classification, the computational burden is much lower, because we now have to train and use only $K$ classifier, compared to ${K \choose 2}$.
The disadvantage of using 1-vs-all is that each classifier now operates on an unbalanced dataset, even if the original classes were roughtly balanced.
Therefore, we need to make sure that the underlying binary classifier $\hat{f}$ is robust to class unbalance.

### Score averaging for multi-class classification

Before showing an example of 1-vs-1 and 1-vs-all classifiers, we have to understand how to compute scores for such classifiers.
Let's consider any score which, like most scores, has been devised with binary classification in mind.
To keep things simple to understand, we might focus on Precision alone.
How can we transform Precision into a score for multi-class classification?

Recall that, when in a binary classification setting:
$$\text{Precision} = \frac{\text{Num True Positives}}{\text{Num Predicted Positive}}$$

Let's, instead, consider a simple 3-class classification problem, with classes A, B, and C.
Note how it's possible to define the Precision for each of the classes:
$$\text{Precision}_k = \frac{\text{Num True } k}{\text{Num Predicted } k} \quad k = A, B, C$$

How to combine these values into a single Precision metric?
There two possible approaches:
* **Macro-averaging** consists in computing the values $\text{Precision}_k$ for all classes $k = A, B, C$ and then returning the average of the three $\text{Precision}$ values:
$$\text{Precision} = \frac{1}{3} \big( \text{Precision}_A + \text{Precision}_B + \text{Precision}_C \big)$$
* **Micro-averaging** consists into moving the sum into the numerator and denominator of the fraction:
$$\text{Precision} = \frac{\text{Num True } A + \text{Num True } B + \text{Num True } C}{\text{Num Predicted } A + \text{Num Predicted } B + \text{Num Predicted } C}$$

What is the difference between doing a micro-average and a macro-average?
The main one consists in robustness to unbalanced class: **when we have unbalanced classes, in fact, it's recommended to use micro-averaging.**
To understand why, consider the following example with confusion matrix:

|Prediction↓ / Real Class→ |   A |   B |   C | Total predicted |
|--------------------------|-----|-----|-----|-----------------|
|A                         | $10$| $20$| $20$| 50              |
|B                         |  $1$| $10$|  $1$| 12              |
|C                         |  $1$|  $1$| $10$| 12              |
|––––––––––––––––––––––––––|–––––|–––––|–––––|–––––––––––––––––|
| **Total of real class:** |   12|   32|   32|                 |

If we use macro-averaging, then, the high precision on classes $B$ and $C$ will "mask" the poor performance on class $A$:

$$
\begin{align}
    \text{Precision}_A &= \frac{10}{50} = 0.2 \\
    \text{Precision}_B &= \frac{10}{12} = 0.83 \\
    \text{Precision}_C &= \frac{10}{12} = 0.83 \\
    \text{Precision}   &= \frac{1}{3}(0.2 + 0.83 + 0.83) = 0.62
\end{align}
$$

Using, micro-averaging, on the other hand, we have:

$$\text{Precision} = \frac{10 + 10 + 10}{12 + 32 + 32} = 0.39$$

As we can see, the **micro-averaged score is more influenced by the low precision the classifier achieves on class** $A$.

Although we used Precision in this example, because of its simplicity, the same reasoning applies to other scores.
For example, the **F1-score can be micro- or macro-averaged**, and `sklearn`'s `f1_score` has an `average=` parameter which lets us specify which type of average we want.

### Example 1-vs-1 and 1-vs-all, micro-average and macro-average

In [65]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.metrics import f1_score

In [66]:
d = load_digits()
X = d['data']
y = d['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0, stratify=y)

In [67]:
model1vs1 = OneVsOneClassifier(LogisticRegression(solver='lbfgs', max_iter=10000)).fit(X_train, y_train)
model1vsRest = OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=10000)).fit(X_train, y_train)

In [68]:
f1_score(y_test, model1vs1.predict(X_test), average='macro')

0.9844516696293155

In [69]:
f1_score(y_test, model1vs1.predict(X_test), average='micro')

0.9844444444444445

In [70]:
f1_score(y_test, model1vsRest.predict(X_test), average='macro')

0.9557228235416941

In [71]:
f1_score(y_test, model1vsRest.predict(X_test), average='micro')

0.9555555555555556